In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers,regularizers
from tensorflow.keras.layers import Add,Conv2D,BatchNormalization,Flatten,MaxPooling2D,AveragePooling2D,Dense,Input,Activation,ZeroPadding2D
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.initializers import glorot_uniform
from matplotlib.pyplot import imshow
import tensorflow.keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)



In [3]:
import os
try:
  device_name=os.environ['COLAB_TPU_ADDR']
  TPU_ADDRESS='grpc://'+device_name
  print('FOUNT TPU at:{}'.format(TPU_ADDRESS))
except KeyError:
  print('TPU not found')

FOUNT TPU at:grpc://10.17.181.74:8470


In [4]:
tf.test.gpu_device_name()

tf.test.gpu_device_name()

''

In [0]:
def identity_block(X,f,filters,stage,block):
  conv_name_base='res'+str(stage)+block+'_branch'
  bn_name_base='bn'+str(stage)+block+'_branch'
  F1,F2,F3=filters
  X_shortcut=X

  X=Conv2D(F1,(1,1),strides=(1,1),padding='valid',name=conv_name_base+'2a',kernel_initializer=glorot_uniform(seed=0))(X)
  X=BatchNormalization(axis=3,name=bn_name_base+'2a')(X)
  X=Activation('relu')(X)

  X=Conv2D(F2,(f,f),strides=(1,1),padding='same',name=conv_name_base+'2b',kernel_initializer=glorot_uniform(seed=0))(X)
  X=BatchNormalization(axis=3,name=bn_name_base+'2b')(X)
  X=Activation('relu')(X)

  X=Conv2D(F3,(1,1),strides=(1,1),padding='valid',name=conv_name_base+'2c',kernel_initializer=glorot_uniform(seed=0))(X)
  X=BatchNormalization(axis=3,name=bn_name_base+'2c')(X)

  X=Add()([X_shortcut,X])
  X=Activation('relu')(X)

  return X

In [6]:
tf.reset_default_graph()

with tf.Session() as test:
    np.random.seed(1)
    A_prev = tf.placeholder("float", [3, 4, 4, 6])
    X = np.random.randn(3, 4, 4, 6)
    A = identity_block(A_prev, f = 2, filters = [2, 4, 6], stage = 1, block = 'a')
    test.run(tf.global_variables_initializer())
    out = test.run([A], feed_dict={A_prev: X, K.learning_phase(): 0})
    print("out = " + str(out[0][1][1][0]))

Instructions for updating:
Colocations handled automatically by placer.
out = [0.94822997 0.         1.1610144  2.747859   0.         1.36677   ]


In [0]:
def convolutional_block(X,f,filters,stage,block,s=2):
  conv_name_base='res'+str(stage)+block+'_branch'
  bn_name_base='bn'+str(stage)+block+'_branch'

  F1,F2,F3=filters
  X_shortcut=X

  X=Conv2D(F1,(1,1),strides=(s,s),name=conv_name_base+'2a',kernel_initializer=glorot_uniform(seed=0))(X)
  X=BatchNormalization(axis=3,name=bn_name_base+'2a')(X)
  X=Activation('relu')(X)

  X=Conv2D(F2,(f,f),strides=(1,1),padding='same',name=conv_name_base+'2b',kernel_initializer=glorot_uniform(seed=0))(X)
  X=BatchNormalization(axis=3,name=bn_name_base+'2b')(X)
  X=Activation('relu')(X)

  X=Conv2D(F3,(1,1),strides=(1,1),padding='valid',name=conv_name_base+'2c',kernel_initializer=glorot_uniform(seed=0))(X)
  X=BatchNormalization(axis=3,name=bn_name_base+'2c')(X)

  X_shortcut=Conv2D(F3,(1,1),strides=(s,s),padding='valid',name=conv_name_base+'1',kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
  X_shortcut=BatchNormalization(axis=3,name=bn_name_base+'1')(X_shortcut)

  X=Add()([X_shortcut,X])
  X=Activation('relu')(X)

  return X




In [8]:
tf.reset_default_graph()

with tf.Session() as test:
    np.random.seed(1)
    A_prev = tf.placeholder("float", [3, 4, 4, 6])
    X = np.random.randn(3, 4, 4, 6)
    A = convolutional_block(A_prev, f = 2, filters = [2, 4, 6], stage = 1, block = 'a')
    test.run(tf.global_variables_initializer())
    out = test.run([A], feed_dict={A_prev: X, K.learning_phase(): 0})
    print("out = " + str(out[0][1][1][0]))

out = [0.09018463 1.2348979  0.46822023 0.03671762 0.         0.65516603]


In [0]:
def resnet50(input_shape=(64,64,3),classes=6):
  X_input=Input(input_shape)

  X=ZeroPadding2D((3,3))(X_input)

  #stage 1
  X=Conv2D(64,(7,7),strides=(2,2),name='conv1',kernel_initializer=glorot_uniform(seed=0))(X)
  X=BatchNormalization(axis=3,name='bn1')(X)
  X=Activation('relu')(X)
  X=MaxPooling2D((3,3),strides=(2,2))(X)

  ## Stage 2
  X=convolutional_block(X,f=3,filters=[64,64,256],stage=2,block='a',s=1)
  X=identity_block(X,f=3,filters=[64,64,256],stage=2,block='b')
  X=identity_block(X,3,filters=[64,64,256],stage=2,block='c')

  ## Stage 3
  X=convolutional_block(X,f=3,filters=[128,128,512],stage=3,block='a',s=2)
  X=identity_block(X,f=3,filters=[128,128,512],stage=3,block='b')
  X=identity_block(X,f=3,filters=[128,128,512],stage=3,block='c')
  X=identity_block(X,f=3,filters=[128,128,512],stage=3,block='d')

  ## Stage 4
  X=convolutional_block(X,f=3,filters=[256,256,1024],stage=4,block='a',s=2)
  X=identity_block(X,3,filters=[256,256,1024],stage=4,block='b')
  X=identity_block(X,3,filters=[256,256,1024],stage=4,block='c')
  X=identity_block(X,3,filters=[256,256,1024],stage=4,block='d')
  X=identity_block(X,3,filters=[256,256,1024],stage=4,block='e')
  X=identity_block(X,3,filters=[256,256,1024],stage=4,block='f')

  ## Stage 5
  X=convolutional_block(X,f=3,filters=[512,512,2048],stage=5,block='a',s=2)
  X=identity_block(X,3,filters=[512,512,2048],stage=5,block='b')
  X=identity_block(X,3,filters=[512,512,2048],stage=5,block='c')
  X=identity_block(X,3,filters=[512,512,2048],stage=5,block='d')

  ## Avgpool

  X=AveragePooling2D(pool_size=(2,2),name='avg_pool')(X)

  X=Flatten()(X)
  X=Dense(classes,activation='softmax',name='fc'+str(classes),kernel_initializer=glorot_uniform(seed=0),kernel_regularizer=regularizers.l2(0.2))(X)
  

  model=Model(inputs=X_input,outputs=X,name='resnet50')

  return model
  

In [0]:
from numpy import load

In [0]:
train_images=load('train_images.npy')
test_images=load('test_images.npy')
train_labels=load('train_labels.npy')

In [12]:
train_images.shape

(1821, 224, 224, 3)

In [0]:
X_train=train_images[0:1600,:,:,:]
X_test=train_images[1600:,:,:,:]
Y_train=train_labels[0:1600,:]
Y_test=train_labels[1600:,:]

In [0]:
model=resnet50(input_shape=(224,224,3),classes=4)

In [0]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])


In [54]:
tpu_model = tf.contrib.tpu.keras_to_tpu_model(
    model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)))

INFO:tensorflow:Querying Tensorflow master (grpc://10.17.181.74:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 3463753295856895594)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 11660470434348942070)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 3002986208143163510)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 3023957146966705369)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 4586319863794119750)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/

In [67]:
tpu_model.fit(X_train,Y_train,batch_size=16,epochs=10)

Epoch 1/10
1600/1600 [==============================] - 7s 5ms/sample - loss: 0.1391 - acc: 0.9725
Epoch 2/10
1600/1600 [==============================] - 7s 5ms/sample - loss: 0.1118 - acc: 0.9775
Epoch 3/10
1600/1600 [==============================] - 7s 4ms/sample - loss: 0.0602 - acc: 0.9894
Epoch 4/10
1600/1600 [==============================] - 7s 5ms/sample - loss: 0.0438 - acc: 0.9925
Epoch 5/10
1600/1600 [==============================] - 7s 4ms/sample - loss: 0.0341 - acc: 0.9944
Epoch 6/10
1600/1600 [==============================] - 7s 4ms/sample - loss: 0.0395 - acc: 0.9931
Epoch 7/10
1600/1600 [==============================] - 7s 5ms/sample - loss: 0.0188 - acc: 0.9987
Epoch 8/10
1600/1600 [==============================] - 7s 4ms/sample - loss: 0.0131 - acc: 0.9994
Epoch 9/10
1600/1600 [==============================] - 7s 5ms/sample - loss: 0.0158 - acc: 0.9987
Epoch 10/10
1600/1600 [==============================] - 7s 5ms/sample - loss: 0.0120 - acc: 0.9987


In [68]:
pred=tpu_model.evaluate(X_test,Y_test)
print()
print("Loss"+str(pred[0]))
print('Test Accuracy'+str(pred[1]))

221/221 [==============================] - 1s 3ms/sample - loss: 0.4646 - acc: 0.8981

Loss0.4645833753352791
Test Accuracy0.8981482


In [0]:
cpu_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_5 (ZeroPadding2D (None, 230, 230, 3)  0           input_6[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        zero_padding2d_5[0][0]           
__________________________________________________________________________________________________
bn1 (BatchNormalizationV1)      (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation